In [1]:
# Do not reload
import sys, os
sys.path.append("../")
os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
from src.config import DATA_PATH
from src.common import input_keys
from src.generate.generators import generate_ip_space, generate_bot_entries, generate_human_entries, generate_dataset

%load_ext autoreload
%autoreload 2

In [3]:
sample = pd.read_csv(DATA_PATH / 'Sample Data for Candidate Assessment.csv', usecols=input_keys.USECOLS)

In [4]:
sample.head()

,REQUEST_TIME,DEVICE_IP,DEVICE_IFA,DEVICE_LANGUAGE,GEO_CURRENT_CITY
0,2022-10-19T15:51:07.387Z,5.28.186.193,741b39aa-3cf6-4fdf-863c-4ca8ab11102f,he,Petah Tiqwa
1,2022-10-19T15:50:05Z,5.28.177.198,5b932c2d-a3b9-4220-9b28-aa4d93c57a59,he,Givatayim
2,2022-10-19T15:56:35.587Z,5.28.186.210,96aa31a6-3ec3-4bab-b33d-ddcb164312e7,en,Bat Yam
3,2022-10-19T15:50:31.205Z,5.28.185.0,AADvyU7EZrYAAC9tt5XFAA,en,Givatayim
4,2022-10-19T15:51:30.363Z,5.28.185.3,AADwHU7GFEoAAA8i1vj7-Q,ru,Givatayim


In [5]:
sample.nunique()

REQUEST_TIME        489
DEVICE_IP           347
DEVICE_IFA          391
DEVICE_LANGUAGE       9
GEO_CURRENT_CITY     36
dtype: int64

# Generate sampling space for attributes: IP, Language, City

In [6]:
IP_SPACE = generate_ip_space(2000)
LANG_SPACE, LANG_P = sample[input_keys.LANG_KEY].value_counts().index, sample[input_keys.LANG_KEY].value_counts(True).values
CITY_SPACE, CITY_P = sample[input_keys.CITY_KEY].value_counts().index, sample[input_keys.CITY_KEY].value_counts(True).values

# Define the samples generation process

The pipeline is the following:
1) Define mass distribution over 24 hours (apply sleeping windows for human entries)
2) Sample random number of total entries a day based on distribution
3) Sample entries in each hour:
    - Take desired number of entries for an hour and sample request time randomly
    - For bot entries can also perform permutations by sampling big number of events in a minute and by sampling hour in equal intervals
    - For bot entries noise in selection of marginal cities or languages can be introduced

In [7]:
data = generate_dataset(
    np.random.randint(2000, 4000), IP_SPACE, LANG_SPACE, LANG_P, CITY_SPACE, CITY_P, input_keys
)
test_data = generate_dataset(
    np.random.randint(400, 600), IP_SPACE, LANG_SPACE, LANG_P, CITY_SPACE, CITY_P, input_keys
)
api_data = generate_dataset(
    np.random.randint(10, 20), IP_SPACE, LANG_SPACE, LANG_P, CITY_SPACE, CITY_P, input_keys
)

In [8]:
data.head()

,REQUEST_TIME,DEVICE_IP,DEVICE_IFA,GEO_CURRENT_CITY,DEVICE_LANGUAGE,GENERATED_AS_BOT
0,2022-12-21 02:30:48.180924,15.99.54.187,8e225a93-366f-44c8-9588-30f79c1910ae,Ramat Gan,en,0
1,2022-12-21 12:09:49.570128,15.99.54.187,8e225a93-366f-44c8-9588-30f79c1910ae,Ramat Gan,en,0
2,2022-12-21 12:59:56.963635,15.99.54.187,8e225a93-366f-44c8-9588-30f79c1910ae,Ramat Gan,en,0
3,2022-12-21 12:04:35.182955,15.99.54.187,8e225a93-366f-44c8-9588-30f79c1910ae,Ramat Gan,en,0
4,2022-12-21 12:18:46.331838,15.99.54.187,8e225a93-366f-44c8-9588-30f79c1910ae,Ramat Gan,en,0


In [9]:
data.to_csv(DATA_PATH / 'generated_ad_data.csv.gz', sep=';', compression='gzip', index=False)
test_data.to_csv(DATA_PATH / 'generated_ad_data_test.csv.gz', sep=';', compression='gzip', index=False)
api_data.to_csv(DATA_PATH / 'test_api.csv', index=False)